<a href="https://colab.research.google.com/github/choi-yh/Algorithm/blob/master/8_5_HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Kruskal II 알고리즘으로 아래 네트워크의 최소비용신장트리를 구하시오.

In [1]:
# 그래프를 정의한다.

g = {}
g['A'] = {}
g['B'] = {}
g['C'] = {}
g['D'] = {}
g['E'] = {}
g['F'] = {}
g['G'] = {}
g['A']['B'] = 3
g['A']['C'] = 17
g['A']['D'] = 6
g['B']['A'] = 3
g['B']['D'] = 5
g['B']['G'] = 12
g['C']['A'] = 17
g['C']['E'] = 10
g['C']['F'] = 8
g['D']['A'] = 6
g['D']['B'] = 5
g['D']['E'] = 9
g['E']['C'] = 10
g['E']['D'] = 9
g['E']['F'] = 4
g['E']['G'] = 2
g['F']['C'] = 8
g['F']['E'] = 4
g['F']['G'] = 14
g['G']['B'] = 12
g['G']['E'] = 2
g['G']['F'] = 14

g

{'A': {'B': 3, 'C': 17, 'D': 6},
 'B': {'A': 3, 'D': 5, 'G': 12},
 'C': {'A': 17, 'E': 10, 'F': 8},
 'D': {'A': 6, 'B': 5, 'E': 9},
 'E': {'C': 10, 'D': 9, 'F': 4, 'G': 2},
 'F': {'C': 8, 'E': 4, 'G': 14},
 'G': {'B': 12, 'E': 2, 'F': 14}}

In [0]:
# 오름차순 정리용 minHeap 사용

class minHeap:
    def __init__(self):
        self.x = [None]
        
    def _exchange(self, i, j):
        if self.x[i] > self.x[j]:
            self.x[i], self.x[j] = self.x[j], self.x[i]
        
    def push(self, item):
        self.x.append(item)
        cidx = len(self.x) - 1
        pidx = cidx // 2
        while pidx >= 1:
            self._exchange(pidx, cidx)
            cidx = pidx
            pidx = cidx // 2
            
    def pop(self):
        _tmp = self.x[1]
        self.x[1], self.x[-1] = self.x[-1], self.x[1]
        self.x = self.x[:-1]
        self.heapify()
        return _tmp
    
            
    def _child(self, pidx):
        n = len(self.x)
        lcidx = pidx * 2
        rcidx = pidx * 2 + 1
        if rcidx < n:
            return lcidx, rcidx
        elif lcidx < n and rcidx >= n:
            return lcidx, None
        else:
            return None, None
            
    def heapify(self):
        for pidx in range(1, len(self.x)+1):
            lcidx, rcidx = self._child(pidx)
            if lcidx != None and rcidx != None:
                if self.x[lcidx] > self.x[rcidx]:
                    self._exchange(pidx, rcidx)
                else:
                    self._exchange(pidx, lcidx)
            elif lcidx != None and rcidx == None:
                self._exchange(pidx, lcidx)
        return self.x
    
        
    def print(self):
        print(self.x)


In [3]:
import copy
        
class MST:
    def __init__(self, graph):
        def _edges(graph):
            # 그래프에서 노드를 꺼내면서 [거리, 노드1, 노드2] 형태로 만든다.
            edges = []
            for key, value in graph.items():
                for item in list(value.keys()):
                    edges.append([value[item], key, item])
            return edges
        
        def _addEdge(graph):
        # 노드를 cost를 기준으로 minHeap에서 pop하여 오름차순 정렬한다.
            x = minHeap()
            for i in range(len(self.edges)): # edge들을 minHeap에 넣는다.
                x.push(self.edges[i])

            add_list = []
            for j in range(1, len(x.x)):
                tmp = x.pop()
                if [tmp[0], tmp[2], tmp[1]] not in add_list:
                    add_list.append(tmp)
            return add_list
        
        self.graph = graph
        self.edges = _edges(graph)
        self.addEdge = _addEdge(graph)
        
    # kruskal2를 이용해서 엣지를 추가해갈때 사이클을 그때그때 확인해줘야한다. => 사이클 확인하는 메서드 필요.
    # dfs 이용해서 사이클 찾기
    # 사이클의 구현은 Uniion Find Algorithm을 이용하여 구현할 수 있다.
    def _cycle(self, graph, start):
        stack = []
        visit = set()
        
        stack.append(start)
        while len(stack) > 0:
            curNode = stack.pop(-1)
            # dfs 알고리즘
            # Step 2: pop한 노드가 가본 곳이 아니면 노드의 이웃노드 중 방문하지 않은 노드집합을 스택에 푸시하고 팝한 노드를 방문 처리한다.
            # Step 3: pop한 노드가 이미 가본 곳이면 방문처리할 필요가 없으나 싸이클이 존재하는 것이므로 싸이클이 존재함을 알려준다.
            if curNode not in visit:
                non_visit = set(graph[curNode].keys())
                for item in non_visit-visit:
                    stack.append(item)
                visit.add(curNode)
            else:
                return True
        return False
        
    def kruskal2(self):
        mst = {}
        for edge in list(self.graph.keys()):
            mst[edge] = {}
        
        n_edge = 0
        while n_edge != len(self.graph)-1:
            item = self.addEdge.pop(0)
            tmp = copy.deepcopy(mst) # 사이클이 생겼을 경우, 추가한 간선을 취소
            
            mst[item[1]][item[2]] = item[0]
            mst[item[2]][item[1]] = item[0]
            if self._cycle(mst, item[1]) == True:
                mst = tmp
            else:
                n_edge += 1
        return mst
        

a = MST(g)
print(a.kruskal2())

{'A': {'B': 3}, 'B': {'A': 3, 'D': 5}, 'C': {'F': 8}, 'D': {'B': 5, 'E': 9}, 'E': {'G': 2, 'F': 4, 'D': 9}, 'F': {'E': 4, 'C': 8}, 'G': {'E': 2}}
